In [1]:
import pennylane as qml
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
from QuSO_utils import *

In [2]:
import pickle

with open('data/QAOA_data.pkl', 'rb') as f:
    QAOA_data = pickle.load(f)

costs = np.load("data/costs.npy")

# Functions for cost readout

In [4]:
import numpy as np
from scipy.optimize import curve_fit

def qpe_probability_function(y, theta, M):
    y = np.array(y, dtype=np.float64)
    numerator = 1 - np.cos(2 * np.pi * (y - theta * M))
    denominator = 1 - np.cos((2 * np.pi / M) * (y - theta * M))
    # When denominator is close to zero, use the limiting value (which gives a ratio of M^2).
    ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)
    P = ratio / (M**2)
    return P

def qae_probability_function(y, theta, M):
    term1 = qpe_probability_function(y, theta, M)
    term2 = qpe_probability_function(y, -theta, M)
    return 0.5 * (term1 + term2)

def estimate_theta_qae(probabilities):
    probabilities = np.array(probabilities, dtype=np.float64)
    M = len(probabilities)  # Number of states (M = 2^n).
    y_vals = np.arange(M)
    
    max_index = np.argmax(probabilities)
    initial_guess = (max_index / M)
    
    # Define narrow bounds around the initial guess.
    lower_bound = max(0, initial_guess - 0.5 / M)
    upper_bound = min(1, initial_guess + 0.5 / M)
    bounds = ([lower_bound], [upper_bound])
    
    # Define the model function with theta as the only free parameter.
    def model(y, theta):
        return qae_probability_function(y, theta, M)
    
    # Attempt two fits (starting at the lower and upper bounds) to help avoid local minima.
    try:
        popt1, pcov1 = curve_fit(model, y_vals, probabilities, p0=[lower_bound], bounds=bounds)
    except Exception as e:
        popt1, pcov1 = (np.array([np.nan]), np.array([[np.inf]]))
        
    try:
        popt2, pcov2 = curve_fit(model, y_vals, probabilities, p0=[upper_bound], bounds=bounds)
    except Exception as e:
        popt2, pcov2 = (np.array([np.nan]), np.array([[np.inf]]))
    
    var1 = pcov1[0, 0] if np.isfinite(pcov1[0, 0]) else np.inf
    var2 = pcov2[0, 0] if np.isfinite(pcov2[0, 0]) else np.inf
    
    if var1 < var2:
        theta_est = popt1[0]
    else:
        theta_est = popt2[0]
    
    return theta_est

In [5]:
def qae_full_amplitudes(a, n_phase):
    M = 2 ** n_phase
    theta = np.arcsin(a) / np.pi  
    theta_prime = np.arcsin(a)  

    c_plus  = (np.sin(theta_prime) - 1j*np.cos(theta_prime)) / np.sqrt(2)
    c_minus = (np.sin(theta_prime) + 1j*np.cos(theta_prime)) / np.sqrt(2)
    
    def A_QPE(y, phi):
        k = np.arange(M)
        return np.sum(np.exp(-2j * np.pi * k * (y/M - phi))) / M

    A_plus  = np.array([A_QPE(y,  theta) for y in range(M)])
    A_minus = np.array([A_QPE(y, -theta) for y in range(M)])
    
    amp_good = (c_plus * A_plus + c_minus * A_minus) / np.sqrt(2)
    amp_bad  = (1j * c_plus * A_plus - 1j * c_minus * A_minus) / np.sqrt(2)
    
    full_state = np.zeros(2 * M, dtype=np.complex128)
    for y in range(M):
        full_state[y * 2 + 0] = amp_bad[y]
        full_state[y * 2 + 1] = amp_good[y]
    
    return full_state

In [6]:
def compute_qae_amplitudes(configs, amplitudes, n_phase):
    qae_amplitudes = []
    for i, config in enumerate(configs):
        qae_amplitudes.append(qae_full_amplitudes(amplitudes[i], n_phase))

    return qae_amplitudes

In [7]:
import itertools

all_configs = [list(bits) for bits in itertools.product([0, 1], repeat=6)]

# Probability Distribution

In [8]:
def QuSo_probs(n_phase_qubits=3, depth=1):
    qae_amplitudes = compute_qae_amplitudes(all_configs, costs, n_phase_qubits)
    
    config_wires = [f"config({i})" for i in range(6)]
    phase_wires = [f"p({i})" for i in range(n_phase_qubits)]
    data_dummy_wire = ["data"]
    
    wires = config_wires + phase_wires + data_dummy_wire
    dev = qml.device("lightning.qubit", wires=wires)
    
    sin_hamiltonian = get_func_hamiltonian(qae_func, -1, phase_wires)
    
    def mixer_layer(beta, wires):
        for wire in wires:
            qml.RX(-2*beta, wires=wire)
    
    def dummy_cost_layer(gamma):
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        phase_application(gamma, sin_hamiltonian)
        for i, config in enumerate(all_configs):
            qml.adjoint(qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config))
    
    def qaoa_layer(gamma, beta):
        dummy_cost_layer(gamma)
        mixer_layer(beta, config_wires)
    
    def qaoa_circuit(params, wires, depth):
        for wire in wires:
            qml.Hadamard(wires=wire)
        qml.layer(qaoa_layer, depth, params[0], params[1])
    
    @qml.qnode(dev)
    def cost_function(params):
        qaoa_circuit(params, config_wires, depth)
        
        return qml.probs(wires=config_wires)
    
    params = QAOA_data[depth][1][-1]
    probs = cost_function(params)

    return probs

In [ ]:
import multiprocessing

def compute_quso_prob(n_phase_qubits, depth):
    result = QuSo_probs(n_phase_qubits, depth)
    print(f"n_phase_qubits={n_phase_qubits}, depth={depth} done!")
    np.save(f"data/probs_phase_{n_phase_qubits}_depth_{depth}.npy", result)
    return (n_phase_qubits, depth, result)

n_phase_qubits_list = range(2, 6)
depths = [1, 2, 3, 4, 5]  # qaoa depth

# Create input arguments for parallel execution
args = [(n, depth) for n in n_phase_qubits_list for depth in depths]

# Use a single multiprocessing pool
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    results = pool.starmap(compute_quso_prob, args)

# Organize results into a dictionary
quso_probs = {depth: [] for depth in depths}
for n_phase_qubits, depth, result in results:
    quso_probs[depth].append(result)

Polynomial Expression:Polynomial Expression:Polynomial Expression:   1.73205080756888*x0*x1 - 0.866025403784439*x0 - 0.866025403784438*x11.73205080756888*x0*x1 - 0.866025403784439*x0 - 0.866025403784438*x11.73205080756888*x0*x1 - 0.866025403784439*x0 - 0.866025403784438*x1


Polynomial Expression: 1.73205080756888*x0*x1 - 0.866025403784439*x0 - 0.866025403784438*x1
Polynomial Expression: 1.73205080756888*x0*x1 - 0.866025403784439*x0 - 0.866025403784438*x1
Polynomial Expression:Polynomial Expression:  Polynomial Expression: 1.3228756555323*x0*x1 + 0.626980168831352*x0*x2 - 0.974927912181824*x0 + 0.240787309403764*x1*x2 - 0.78183148246803*x1 - 0.433883739117558*x21.3228756555323*x0*x1 + 0.626980168831352*x0*x2 - 0.974927912181824*x0 + 0.240787309403764*x1*x2 - 0.78183148246803*x1 - 0.433883739117558*x2

1.3228756555323*x0*x1 + 0.626980168831352*x0*x2 - 0.974927912181824*x0 + 0.240787309403764*x1*x2 - 0.78183148246803*x1 - 0.433883739117558*x2
Polynomial Expression:Polynomial Expression: 

In [ ]:
import pickle 

with open('data/quso_probs.pkl', 'wb') as f:
    pickle.dump(quso_probs, f)

In [ ]:
bitstrings = [''.join(seq) for seq in itertools.product('01', repeat=6)]
sorted_indices = sorted(range(len(costs)), key=lambda i: costs[i])
sorted_bitstrings = [bitstrings[i] for i in sorted_indices]

for depth in depths:
    plt.figure(figsize=(10, 5))
    plt.style.use('default')
    for i, n_phase_qubits in enumerate(n_phase_qubits_list):
        probs_ = quso_probs[depth][i]
        sorted_probs_ = [probs_[j] for j in sorted_indices]
        plt.plot(range(len(sorted_probs_)), sorted_probs_, label=f"{n_phase_qubits} phase qubits")
    qaoa_probs = QAOA_data[depth][2]
    plt.plot(range(len(qaoa_probs)), [qaoa_probs[j] for j in sorted_indices], linestyle='--', color='black', label="QAOA")
    plt.ylabel("Probability")
    plt.xlabel("Bitstrings")
    plt.xticks(ticks=range(len(sorted_bitstrings)), labels=sorted_bitstrings, rotation=90)
    plt.title(f"Bar Plot Sorted by Costs, Depth={depth}")
    # plt.xticks(ticks=range(2 ** num_qaoa_wires), labels=bitstrings, rotation=90)  # Add bitstrings as x labels
    plt.legend()
    plt.show()

# QuSO Cost Evaluation

In [ ]:
def quso_cost(n_phase_qubits, depth):
    
    qae_amplitudes = compute_qae_amplitudes(all_configs, costs, n_phase_qubits)
    
    config_wires = [f"config({i})" for i in range(6)]
    phase_wires = [f"p({i})" for i in range(n_phase_qubits)]
    data_dummy_wire = ["data"]
    
    wires = config_wires + phase_wires + data_dummy_wire
    dev = qml.device("lightning.qubit", wires=wires)
    
    sin_hamiltonian = get_func_hamiltonian(qae_func, -1, phase_wires)
    
    def mixer_layer(beta, wires):
        for wire in wires:
            qml.RX(-2*beta, wires=wire)
    
    def dummy_cost_layer(gamma):
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        phase_application(gamma, sin_hamiltonian)
        for i, config in enumerate(all_configs):
            qml.adjoint(qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config))
    
    def qaoa_layer(gamma, beta):
        dummy_cost_layer(gamma)
        mixer_layer(beta, config_wires)
    
    def qaoa_circuit(params, wires, depth):
        for wire in wires:
            qml.Hadamard(wires=wire)
        qml.layer(qaoa_layer, depth, params[0], params[1])

    @qml.qnode(dev)
    def cost_function(params):
        qaoa_circuit(params, config_wires, depth)
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        
        return qml.probs(wires=config_wires + phase_wires)
    
    def cost_function_quso(params_flat):
        params = params_flat.reshape(2, -1)
        probs = cost_function(params)
    
        cost_curvefit = 0
        for i in range(2**len(config_wires)):
            probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
            probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
            a = np.sin(np.pi*estimate_theta_qae(probs_qae_i))
            cost_curvefit += probs_i * a
    
        cost_maxprob = 0
        bit_strings = [f"0.{x:0{n_phase_qubits}b}" for x in range(2**len(phase_wires))]
        for i in range(2**len(config_wires)):
            probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
            probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
            a = np.sin(bin2dec(bit_strings[probs_qae_i.tolist().index(max(probs_qae_i))])*np.pi)
            cost_maxprob += probs_i * a
    
        return cost_curvefit, cost_maxprob

    params = QAOA_data[depth][1][-1]
    
    return cost_function_quso(np.array(params))

In [ ]:
import multiprocessing
def compute_quso_cost(n_phase_qubits, depth):
    result = quso_cost(n_phase_qubits, depth)
    print(f"n_phase_qubits={n_phase_qubits}, depth={depth} done!")
    return (n_phase_qubits, depth, result)

n_phase_qubits_list = range(2, 6)
depths = [1, 2, 3, 4, 5] 

# Create input arguments for parallel execution
args = [(n, depth) for n in n_phase_qubits_list for depth in depths]

# Use a single multiprocessing pool
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    cost_results = pool.starmap(compute_quso_cost, args)

# Organize results into a dictionary
quso_cost_curvefit = {n_phase_qubits: [] for n_phase_qubits in n_phase_qubits_list}
quso_cost_maxprob = {n_phase_qubits: [] for n_phase_qubits in n_phase_qubits_list}
for n_phase_qubits, depth, result in cost_results:
    quso_cost_curvefit[n_phase_qubits].append(result[0])
    quso_cost_maxprob[n_phase_qubits].append(result[1])

/tmp/ipykernel_3792146/1026330035.py:9: RuntimeWarning: invalid value encountered in divide
  ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)
/tmp/ipykernel_3792146/1026330035.py:9: RuntimeWarning: divide by zero encountered in divide
  ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)


n_phase_qubits=4, depth=4 done!


/tmp/ipykernel_3792146/1026330035.py:9: RuntimeWarning: invalid value encountered in divide
  ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)
/tmp/ipykernel_3792146/1026330035.py:9: RuntimeWarning: divide by zero encountered in divide
  ratio = np.where(np.abs(denominator) < 1e-8, M**2, numerator / denominator)


n_phase_qubits=4, depth=5 done!
n_phase_qubits=5, depth=1 done!
n_phase_qubits=5, depth=2 done!


In [ ]:
# Organize results into a dictionary
quso_cost_curvefit = {n_phase_qubits: [] for n_phase_qubits in n_phase_qubits_list}
quso_cost_maxprob = {n_phase_qubits: [] for n_phase_qubits in n_phase_qubits_list}
for n_phase_qubits, depth, result in cost_results:
    quso_cost_curvefit[n_phase_qubits].append(result[0])
    quso_cost_maxprob[n_phase_qubits].append(result[1])

In [ ]:
cost_min = np.min(costs).real
for n_phase_qubits in quso_cost_curvefit.keys():
    print(n_phase_qubits)
    ar_curvefit = np.array([quso_cost_curvefit[n_phase_qubits][i]/cost_min for i in range(5)])
    ar_maxprob = np.array([quso_cost_maxprob[n_phase_qubits][i]/cost_min for i in range(5)])
    plt.plot(range(1, 6), ar_curvefit, label=n_phase_qubits, marker='*', linestyle='dotted')
    # plt.plot(range(1, 6), ar_maxprob, label=n_phase_qubits, marker='*', linestyle='-')
qaoa_ar = np.array([0.31188207467553253, 0.4255929576000717, 0.5451693449032563, 0.6437182806302651, 0.7212834489117156])
#plt.plot(range(1, 6), 1/qaoa_ar, label="QAOA", marker='*', linestyle='dotted')
plt.legend()
plt.show()

In [ ]:
import pickle 

with open('data/quso_cost_curvefit.pkl', 'wb') as f:
    pickle.dump(quso_cost_curvefit, f)

with open('data/quso_cost_maxprob.pkl', 'wb') as f:
    pickle.dump(quso_cost_maxprob, f)

# Cost Landscapes

In [ ]:
def compute_cost_landscape(n_phase_qubits):
    qae_amplitudes = compute_qae_amplitudes(all_configs, costs, n_phase_qubits)
    
    config_wires = [f"config({i})" for i in range(6)]
    phase_wires = [f"p({i})" for i in range(n_phase_qubits)]
    data_dummy_wire = ["data"]
    
    wires = config_wires + phase_wires + data_dummy_wire
    dev = qml.device("lightning.qubit", wires=wires)
    
    sin_hamiltonian = get_func_hamiltonian(qae_func, -1, phase_wires)
    
    def mixer_layer(beta, wires):
        for wire in wires:
            qml.RX(-2*beta, wires=wire)
    
    def dummy_cost_layer(gamma):
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        phase_application(gamma, sin_hamiltonian)
        for i, config in enumerate(all_configs):
            qml.adjoint(qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config))
    
    def qaoa_layer(gamma, beta):
        dummy_cost_layer(gamma)
        mixer_layer(beta, config_wires)
    
    def qaoa_circuit(params, wires, depth):
        for wire in wires:
            qml.Hadamard(wires=wire)
        qml.layer(qaoa_layer, depth, params[0], params[1])
    
    p = 1
    
    @qml.qnode(dev)
    def cost_function(params):
        qaoa_circuit(params, config_wires, p)
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        
        return qml.probs(wires=config_wires + phase_wires)
    
    def cost_function_quso(params_flat):
        params = params_flat.reshape(2, -1)
        probs = cost_function(params)
    
        cost_curvefit = 0
        for i in range(2**len(config_wires)):
            probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
            probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
            a = np.sin(np.pi*estimate_theta_qae(probs_qae_i))
            cost_curvefit += probs_i * a
    
        # cost_maxprob = 0
        # bit_strings = [f"0.{x:0{n_phase_qubits}b}" for x in range(2**len(phase_wires))]
        # for i in range(2**len(config_wires)):
        #     probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
        #     probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
        #     a = np.sin(bin2dec(bit_strings[probs_qae_i.tolist().index(max(probs_qae_i))])*np.pi)
        #     cost_maxprob += probs_i * a
    
        return cost_curvefit# , cost_maxprob

    global compute_cost
    def compute_cost(args):
        i, j, X, Y = args
        return i, j, cost_function_quso(np.array([[Y[i]], [X[j]]]))
    
    X = np.linspace(-np.pi/2, np.pi/2, 50)
    Y = np.linspace(-3*np.pi, 3*np.pi, 50)
    
    cost_landscape = np.zeros((len(Y), len(X)))
    
    args_list = [(i, j, X, Y) for i in range(len(Y)) for j in range(len(X))]
    
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(compute_cost, args_list)

    for i, j, value in results:
        cost_landscape[i, j] = value

    X, Y = np.meshgrid(X, Y)

    return cost_landscape, X, Y

In [ ]:
import pickle 

cost_landscapes = {}
for n_phase_qubits in [2, 3, 4, 5]:
    cost_landscapes[n_phase_qubits] = compute_cost_landscape(n_phase_qubits)
    print(f"Done with {n_phase_qubits} phase qubits!")
    with open('data/cost_landscapes.pkl', 'wb') as f:
        pickle.dump(cost_landscapes, f)

In [ ]:
def compute_cost_curve(n_phase_qubits, depth):
    qae_amplitudes = compute_qae_amplitudes(all_configs, costs, n_phase_qubits)
    
    config_wires = [f"config({i})" for i in range(6)]
    phase_wires = [f"p({i})" for i in range(n_phase_qubits)]
    data_dummy_wire = ["data"]
    
    wires = config_wires + phase_wires + data_dummy_wire
    dev = qml.device("lightning.qubit", wires=wires)
    
    sin_hamiltonian = get_func_hamiltonian(qae_func, -1, phase_wires)
    
    def mixer_layer(beta, wires):
        for wire in wires:
            qml.RX(-2*beta, wires=wire)
    
    def dummy_cost_layer(gamma):
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        phase_application(gamma, sin_hamiltonian)
        for i, config in enumerate(all_configs):
            qml.adjoint(qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config))
    
    def qaoa_layer(gamma, beta):
        dummy_cost_layer(gamma)
        mixer_layer(beta, config_wires)
    
    def qaoa_circuit(params, wires, depth):
        for wire in wires:
            qml.Hadamard(wires=wire)
        qml.layer(qaoa_layer, depth, params[0], params[1])
    
    @qml.qnode(dev)
    def cost_function(params):
        qaoa_circuit(params, config_wires, depth)
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires+data_dummy_wire), control=config_wires, control_values=config)
        
        return qml.probs(wires=config_wires + phase_wires)

    global cost_function_quso
    def cost_function_quso(params_flat):
        params = params_flat.reshape(2, -1)
        probs = cost_function(params)
    
        cost_curvefit = 0
        for i in range(2**len(config_wires)):
            probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
            probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
            a = np.sin(np.pi*estimate_theta_qae(probs_qae_i))
            cost_curvefit += probs_i * a
    
        # cost_maxprob = 0
        # bit_strings = [f"0.{x:0{n_phase_qubits}b}" for x in range(2**len(phase_wires))]
        # for i in range(2**len(config_wires)):
        #     probs_i = np.sum(probs.reshape((2**len(config_wires), -1)), axis=1)[i]
        #     probs_qae_i = probs.reshape((2**len(config_wires), -1))[i, :]/probs_i
        #     a = np.sin(bin2dec(bit_strings[probs_qae_i.tolist().index(max(probs_qae_i))])*np.pi)
        #     cost_maxprob += probs_i * a
    
        return cost_curvefit# , cost_maxprob

    params_list_qaoa = np.array(QAOA_data[depth][1])
    cost_list_quso = []
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        cost_list_quso = pool.map(cost_function_quso, params_list_qaoa)

    return cost_list_quso

In [ ]:
def compute_quso_cost_curve(n_phase_qubits, depth):
    result = compute_cost_curve(n_phase_qubits, depth)
    print(f"n_phase_qubits={n_phase_qubits}, depth={depth} done!")
    return (n_phase_qubits, depth, result)

n_phase_qubits_list = range(2, 6)
depths = [1, 2, 3, 4, 5] 

# Create input arguments for parallel execution
args = [(n, i) for n in n_phase_qubits_list for i in depths]

# Use a single multiprocessing pool
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    cost_curve_results = pool.starmap(compute_quso_cost_curve, args)

# Organize results into a dictionary
cost_curve_curvefit = {}
for n_phase_qubits, depth, result in cost_curve_results:
    cost_curve_curvefit[(n_phase_qubits, depth)] = result

with open('data/cost_curves.pkl', 'wb') as f:
        pickle.dump(cost_curve_curvefit, f)

# QAOA cost curves

In [ ]:
import multiprocessing
import pickle
import numpy as np
import pennylane as qml

# Assumed to be defined globally:
# all_configs, costs, QAOA_data, qae_func, get_func_hamiltonian, phase_application, estimate_theta_qae

def compute_single_cost(n_phase_qubits, depth, params_flat):
    # Compute amplitudes based on n_phase_qubits
    qae_amplitudes = compute_qae_amplitudes(all_configs, costs, n_phase_qubits)
    
    # Define wires and create the device
    config_wires = [f"config({i})" for i in range(6)]
    phase_wires = [f"p({i})" for i in range(n_phase_qubits)]
    data_dummy_wire = ["data"]
    wires = config_wires + phase_wires + data_dummy_wire
    dev = qml.device("lightning.qubit", wires=wires)
    
    # Get the Hamiltonian function for phase rotations
    sin_hamiltonian = get_func_hamiltonian(qae_func, -1, phase_wires)
    
    def mixer_layer(beta, wires):
        for wire in wires:
            qml.RX(-2 * beta, wires=wire)
    
    def dummy_cost_layer(gamma):
        # Apply state-preparations conditioned on each configuration
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires + data_dummy_wire),
                     control=config_wires, control_values=config)
        phase_application(gamma, sin_hamiltonian)
        for i, config in enumerate(all_configs):
            qml.adjoint(
                qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires + data_dummy_wire),
                         control=config_wires, control_values=config)
            )
    
    def qaoa_layer(gamma, beta):
        dummy_cost_layer(gamma)
        mixer_layer(beta, config_wires)
    
    def qaoa_circuit(params, wires, depth):
        # Initialize all wires to Hadamard basis
        for wire in wires:
            qml.Hadamard(wires=wire)
        qml.layer(qaoa_layer, depth, params[0], params[1])
    
    @qml.qnode(dev)
    def cost_function(params):
        qaoa_circuit(params, config_wires, depth)
        # Final state-preparation before measurement
        for i, config in enumerate(all_configs):
            qml.ctrl(qml.StatePrep(qae_amplitudes[i], wires=phase_wires + data_dummy_wire),
                     control=config_wires, control_values=config)
        return qml.probs(wires=config_wires + phase_wires)
    
    def cost_function_quso(params_flat):
        # Reshape parameter array as required
        params = params_flat.reshape(2, -1)
        probs = cost_function(params)
        cost_curvefit = 0
        # Reshape probabilities: rows correspond to config wires, columns to phase wires
        probs_reshaped = probs.reshape((2**len(config_wires), -1))
        for i in range(2**len(config_wires)):
            probs_i = np.sum(probs_reshaped, axis=1)[i]
            # Avoid division by zero
            if probs_i == 0:
                continue
            probs_qae_i = probs_reshaped[i, :] / probs_i
            a = np.sin(np.pi * estimate_theta_qae(probs_qae_i))
            cost_curvefit += probs_i * a
        return cost_curvefit

    return cost_function_quso(params_flat)

# Define ranges for n_phase_qubits and depths
n_phase_qubits_list = range(2, 6)
depths = [1, 2, 3, 4, 5]

# Flatten the parameter evaluations: for each (n_phase_qubits, depth) pair, each parameter set gets its own job.
args = []
for n in n_phase_qubits_list:
    for d in depths:
        for params in QAOA_data[d][1]:
            args.append((n, d, np.array(params)))

# Run all evaluations in parallel using the multiprocessing pool
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
results = pool.starmap(compute_single_cost, args)
pool.close()
pool.join()

# Group results by (n_phase_qubits, depth)
cost_curve_curvefit = {}
for (n, d, _), cost in zip(args, results):
    cost_curve_curvefit.setdefault((n, d), []).append(cost)

# Save the results to a pickle file
with open('data/cost_curves.pkl', 'wb') as f:
    pickle.dump(cost_curve_curvefit, f)

print("All computations are complete and saved.")
